# Intelligent Sampling Evaluation notebook

In [1]:
import sys
import logging

## LOGGING
nblog = open("alibaba_data-evaluation.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import pickle

from sklearn.metrics import f1_score, accuracy_score

import time

import random


from collections import Counter

## Load data

In [3]:
sampled_workload_data = pd.read_csv('experiments/100_001_sampled_workload_data_full.csv', index_col=0)

In [4]:
sampled_workload_data

,inst_id,user,task_name,inst_num,status,start_time,end_time,plan_cpu,plan_mem,plan_gpu,gpu_type,group,workload,inst_name,worker_name,machine_x,gpu_name,cpu_usage,gpu_wrk_util,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,read,write,read_count,write_count,duration,gpu_type_machine_spec,cap_cpu_machine_spec,cap_mem_machine_spec,cap_gpu_machine_spec,machine_y,machine_cpu_iowait_machine_metric,machine_cpu_kernel_machine_metric,machine_cpu_usr_machine_metric,machine_gpu_machine_metric,machine_load_1_machine_metric,machine_net_receive_machine_metric,machine_num_worker_machine_metric,machine_cpu_machine_metric
job_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71712e3c961b5bbdb2ce0440,c5be8813bf90aaf9ca6f02f744ccd95596e9850928a498...,2cbd33b08024,ps,5.0,Terminated,4625696.0,4628399.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,c2e0be7577f427c47d77dd388f5bad181447f0bd21bc9e...,04e8a1d93c8a93a3a0a10466cd2f25132dd14c73293d24...,137e86ce1bc508e64d5fc5ee,/dev/nvidia1,34.465789,7.541667,1.204708,2.723633,0.347158,0.829102,2.068066e+06,3.426616e+05,312.218182,219.376623,2703.0,P100,64.0,512.0,2.0,137e86ce1bc508e64d5fc5ee,0.005480,2.603109,9.889830,45.808696,7.668093,2.620698e+08,5.335775,12.498419
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,1e999417385ceac6fc6908fc833dbc1bf7ccb5ab9e1af7...,3c41010cedf0cc85aa05e197131d20a5684be2c7dd2844...,e43590f20ce91e21cf8f0062,/dev/nvidia0,52.794326,12.049123,1.505760,2.671875,0.445045,0.827148,1.234028e+06,3.724553e+05,264.849650,173.374126,1595.0,P100,64.0,512.0,2.0,e43590f20ce91e21cf8f0062,0.001801,10.529087,28.466644,80.292014,34.026547,1.068209e+08,3.888263,38.997531
b68f25260271d65caccca8e7,5229275f7f92865554f934e97f576ef99db0c079bba59d...,d4d51aca8806,worker,16.0,Terminated,4058070.0,4058321.0,50.0,29.296875,100.0,T4,8bc721404a5d34c9f68829312d324994,bert,f004b4f39329b0c12a6b8de546f95fe477885ed004c856...,db23db95b2af54b7a37b4cc1beb4935c57a5c385d654a8...,f1f4f45a9474a2fd362d76fe,/dev/nvidia0,75.052632,23.727273,1.037900,1.853516,0.445890,0.774414,1.010761e+07,3.781242e+06,2401.913043,590.652174,251.0,T4,96.0,512.0,2.0,f1f4f45a9474a2fd362d76fe,0.002509,2.282917,1.616539,28.132630,4.036872,1.746246e+07,1.537805,3.901965
8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,29.296875,NaN,NaN,02a6709662bff12fea88270e3eb1231d,bert,570f01d774b50c6d0026967d83b1c3403d093ffbe4c474...,3c8e0b3564bf056fbd705402c5d3429e8a89643d4c8d6c...,3c4af9c9b5db2d0a07860338,/dev/nvidia1,92.604240,11.706294,1.481600,2.667969,0.449140,0.829102,2.358233e+06,4.527896e+05,404.843206,271.797909,1595.0,P100,64.0,512.0,2.0,3c4af9c9b5db2d0a07860338,0.002109,1.872507,6.680456,20.784838,5.452154,1.350479e+08,5.147730,8.555072
446cd4921e0f00ab5ce1c2ea,12b9ec70cc7c63ebc7184b8a0c363513a8c3cb19746c74...,2cbd33b08024,ps,2.0,Terminated,5521030.0,5526746.0,600.0,20.000000,NaN,NaN,4ac7f9c48a6913dfa81169f7d6a150fa,bert,a5d7f3cf14cc3baef1b59eab27895e2222f2302735d873...,c34f6c58302ec92032293a469d2cbfaccba65ff97203b6...,d4f54f7e5cf18341942c47ef,/dev/nvidia0,110.590788,13.005300,2.357810,3.056641,0.645605,0.827148,1.037695e+06,1.527555e+05,201.449250,181.084731,5716.0,P100,64.0,512.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e49cae3b33b5c205ed8542b0,5283a27d4f19d767256f97c18880a70d69f4d97ca655a3...,47baba33cffe,ps,1.0,Terminated,5890601.0,5893675.0,600.0,29.296875,NaN,NaN,7389796d26bc0d9c8bae9700a46e88ca,xlnet,8d6b9f3b9ef8539217bc16388fb88e18542c0048677344...,20fb53a63dab2ac5da0e2bc08328ee3835d5b0bf0a78f2...,2988f6d00a08192b50b767f4,/dev/nvidia1,127.340568,81.609635,1.940527,1.995117,1.200031,1.213867,1.941028e+06,1.165604e+06,402.442786,171.301824,

## Load hdbscan results

In [5]:
hdbscan_clusterer = pickle.load(open("experiments/hdbscan_300_power_transform_euclidean.pkl", "rb"))

In [6]:
label_stats = Counter(hdbscan_clusterer.labels_)

label_stats.most_common(10)


[(-1, 24603),
 (8, 12522),
 (9, 11612),
 (25, 7754),
 (23, 7474),
 (1, 5751),
 (12, 5501),
 (3, 3043),
 (13, 2287),
 (20, 2128)]

In [7]:
sampled_workload_data_with_profiles = sampled_workload_data.copy()
sampled_workload_data_with_profiles['profile'] = hdbscan_clusterer.labels_
sampled_workload_data_with_profiles
sampled_workload_data_with_profiles['profile'].value_counts()

-1     24603
 8     12522
 9     11612
 25     7754
 23     7474
 1      5751
 12     5501
 3      3043
 13     2287
 20     2128
 0      2101
 19     2072
 24     1994
 16     1943
 22     1891
 2       836
 15      777
 7       737
 5       735
 4       717
 14      686
 10      648
 18      585
 17      567
 21      395
 6       340
 11      302
Name: profile, dtype: int64

In [8]:
workloads_baseline_info = sampled_workload_data_with_profiles[['gpu_type_machine_spec', 'profile']].copy()
workloads_baseline_info

,gpu_type_machine_spec,profile
job_name,,
71712e3c961b5bbdb2ce0440,P100,9
8aa3a3a479479aecb41ea324,P100,9
b68f25260271d65caccca8e7,T4,0
8aa3a3a479479aecb41ea324,P100,9
446cd4921e0f00ab5ce1c2ea,P100,9
...,...,...
e49cae3b33b5c205ed8542b0,T4,9
23f6e20075e637b0976b4ac8,T4,-1
bf7ee12301073cc75cba61a2,T4,9


## Information from Profiles

### Includes "cluster" -1 removed.

In [9]:
sampled_workload_gpu_v100 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'V100']
sampled_workload_gpu_v100_count = sampled_workload_gpu_v100.groupby('profile').count()

In [10]:
V100_series = sampled_workload_gpu_v100_count['inst_id']
V100_series = V100_series.drop(index=-1).reindex(range(0,26),fill_value=0)
V100_series = V100_series.rename('V100')

In [11]:
sampled_workload_gpu_p100 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'P100']
sampled_workload_gpu_p100_count = sampled_workload_gpu_p100.groupby('profile').count()

In [12]:
P100_series = sampled_workload_gpu_p100_count['inst_id']
P100_series = P100_series.drop(index=-1).reindex(range(0,26),fill_value=0)
P100_series = P100_series.rename('P100')

In [13]:
sampled_workload_gpu_misc = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'MISC']
sampled_workload_gpu_misc_count = sampled_workload_gpu_misc.groupby('profile').count()

In [14]:
misc_series = sampled_workload_gpu_misc_count['inst_id']
misc_series = misc_series.drop(index=-1).reindex(range(0,26),fill_value=0)
misc_series = misc_series.rename('MISC')

In [15]:
sampled_workload_gpu_V100M32 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'V100M32']
sampled_workload_gpu_V100M32_count = sampled_workload_gpu_V100M32.groupby('profile').count()

In [16]:
V100M32_series = sampled_workload_gpu_V100M32_count['inst_id']
V100M32_series = V100M32_series.drop(index=-1).reindex(range(0,26),fill_value=0)
V100M32_series = V100M32_series.rename('V100M32')

In [17]:
sampled_workload_gpu_T4 = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'T4']
sampled_workload_gpu_T4_count = sampled_workload_gpu_T4.groupby('profile').count()

In [18]:
T4_series = sampled_workload_gpu_T4_count['inst_id']
T4_series = T4_series.drop(index=-1).reindex(range(0,26),fill_value=0)
T4_series = T4_series.rename('T4')

In [19]:
sampled_workload_gpu_cpu = sampled_workload_data_with_profiles[sampled_workload_data_with_profiles.gpu_type_machine_spec == 'CPU']
sampled_workload_gpu_cpu_count = sampled_workload_gpu_cpu.groupby('profile').count()

In [20]:
CPU_series = sampled_workload_gpu_cpu_count['inst_id']
CPU_series = CPU_series.drop(index=-1).reindex(range(0,26),fill_value=0)
CPU_series = CPU_series.rename('CPU')

In [21]:
gpu_profiles = pd.DataFrame([V100_series, P100_series, misc_series, V100M32_series, T4_series, CPU_series])

### P(profile | gpu)

In [22]:
gpu_profiles

profile,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
V100,47,74,0,0,0,0,0,4,316,348,9,4,69,0,11,0,0,37,1,21,19,13,7,108,4,138
P100,26,0,0,0,0,0,0,89,7644,7590,298,0,0,0,283,0,0,220,97,680,624,137,151,1046,174,1060
MISC,30,0,0,0,0,0,0,626,3636,3594,269,232,0,3,311,0,0,262,170,1234,1255,221,1722,6249,1796,6482
V100M32,0,0,0,0,0,0,0,2,11,8,3,0,0,0,1,0,0,4,11,11,10,1,2,15,3,18
T4,1991,5677,836,3043,717,735,340,0,864,7,0,64,5432,2284,0,777,1943,0,297,0,74,0,0,0,0,0
CPU,6,0,0,0,0,0,0,16,50,64,69,2,0,0,79,0,0,44,9,125,145,23,9,54,16,54


In [23]:
gpus_profiles_prob = gpu_profiles.copy()
gpus_profiles_prob.loc['V100'] = gpus_profiles_prob.loc['V100']/gpus_profiles_prob.loc['V100'].sum()
gpus_profiles_prob.loc['P100'] = gpus_profiles_prob.loc['P100']/gpus_profiles_prob.loc['P100'].sum()
gpus_profiles_prob.loc['MISC'] = gpus_profiles_prob.loc['MISC']/gpus_profiles_prob.loc['MISC'].sum()
gpus_profiles_prob.loc['V100M32'] = gpus_profiles_prob.loc['V100M32']/gpus_profiles_prob.loc['V100M32'].sum()
gpus_profiles_prob.loc['T4'] = gpus_profiles_prob.loc['T4']/gpus_profiles_prob.loc['T4'].sum()
gpus_profiles_prob.loc['CPU'] = gpus_profiles_prob.loc['CPU']/gpus_profiles_prob.loc['CPU'].sum()

### P(profile | gpu)

In [24]:
gpus_profiles_prob

profile,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
V100,0.038211,0.060163,0.000000,0.000000,0.000000,0.000000,0.000000,0.003252,0.256911,0.282927,0.007317,0.003252,0.056098,0.000000,0.008943,0.00000,0.000000,0.030081,0.000813,0.017073,0.015447,0.010569,0.005691,0.087805,0.003252,0.112195
P100,0.001292,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004424,0.379939,0.377255,0.014812,0.000000,0.000000,0.000000,0.014066,0.00000,0.000000,0.010935,0.004821,0.033799,0.031015,0.006809,0.007505,0.051991,0.008649,0.052687
MISC,0.001068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022284,0.129432,0.127937,0.009576,0.008259,0.000000,0.000107,0.011071,0.00000,0.000000,0.009326,0.006052,0.043927,0.044675,0.007867,0.061299,0.222448,0.063933,0.230742
V100M32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.110000,0.080000,0.030000,0.000000,0.000000,0.000000,0.010000,0.00000,0.000000,0.040000,0.110000,0.110000,0.100000,0.010000,0.020000,0.150000,0.030000,0.180000
T4,0.079383,0.226347,0.033332,0.121327,0.028587,0.029305,0.013556,0.000000,0.034448,0.000279,0.000000,0.002552,0.216578,0.091065,0.000000,0.03098,0.077469,0.000000,0.011842,0.000000,0.002950,0.000000,0.000000,0.000000,0.000000,0.000000
CPU,0.007843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020915,0.065359,0.083660,0.090196,0.002614,0.000000,0.000000,0.103268,0.00000,0.000000,0.057516,0.011765,0.163399,0.189542,0.030065,0.011765,0.070588,0.020915,0.070588


### P(gpu) 
From their article

In [25]:
gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}
gpus = pd.DataFrame.from_dict(gpus_dict, orient='index')
gpus = gpus.rename(columns={0:'Nodes'})

### P(gpu) 

In [26]:
gpus['prob'] = gpus['Nodes']/gpus.Nodes.sum()
gpus

,Nodes,prob
P100,798,0.420664
T4,497,0.261993
MISC,280,0.147601
V100M32,135,0.071165
V100,104,0.054823
CPU,83,0.043753


### P(profile)

In [27]:
profiles = pd.DataFrame.from_dict(label_stats, orient='index').sort_index()
profiles = profiles.rename(columns={0:'profile_counts'})
profiles = profiles.drop(-1)

In [28]:
for ii in profiles.index:
    profiles.loc[ii,'prob'] = (gpus_profiles_prob[ii]*gpus['prob']).sum()
# P_0 = (gpus_profiles_prob[0]*gpus['prob']).sum()

profiles

,profile_counts,prob
0,2101,0.023937
1,5751,0.062599
2,836,0.008733
3,3043,0.031787
4,717,0.007490
5,735,0.007678
6,340,0.003552
7,737,0.007667
8,12522,0.212729
9,11612,0.202519


## GPU given a profile

In [29]:
gpu_given_profile = gpus_profiles_prob.copy()
gpu_given_profile.loc['V100'] = gpus_profiles_prob.loc['V100']*gpus.loc['V100', 'prob']/profiles['prob']
gpu_given_profile.loc['P100'] = gpus_profiles_prob.loc['P100']*gpus.loc['P100', 'prob']/profiles['prob']
gpu_given_profile.loc['MISC'] = gpus_profiles_prob.loc['MISC']*gpus.loc['MISC', 'prob']/profiles['prob']
gpu_given_profile.loc['V100M32'] = gpus_profiles_prob.loc['V100M32']*gpus.loc['V100M32', 'prob']/profiles['prob']
gpu_given_profile.loc['T4'] = gpus_profiles_prob.loc['T4']*gpus.loc['T4', 'prob']/profiles['prob']
gpu_given_profile.loc['CPU'] = gpus_profiles_prob.loc['CPU']*gpus.loc['CPU', 'prob']/profiles['prob']

gpu_given_profile

profile,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
V100,0.087516,0.052689,0.0,0.0,0.0,0.0,0.0,0.023255,0.066210,0.076590,0.028396,0.081776,0.051414,0.00000,0.036943,0.0,0.0,0.126968,0.003093,0.025563,0.023094,0.087366,0.021584,0.065689,0.010936,0.078586
P100,0.022711,0.000000,0.0,0.0,0.0,0.0,0.0,0.242722,0.751317,0.783619,0.441068,0.000000,0.000000,0.00000,0.445857,0.0,0.0,0.354146,0.140734,0.388310,0.355792,0.431907,0.218417,0.298448,0.223157,0.283164
MISC,0.006585,0.000000,0.0,0.0,0.0,0.0,0.0,0.429016,0.089806,0.093244,0.100051,0.559116,0.000000,0.00066,0.123126,0.0,0.0,0.105984,0.061981,0.177077,0.179818,0.175082,0.625924,0.448050,0.578823,0.435131
V100M32,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.185647,0.036799,0.028112,0.151129,0.000000,0.000000,0.00000,0.053622,0.0,0.0,0.219158,0.543197,0.213796,0.194066,0.107302,0.098464,0.145668,0.130954,0.163660
T4,0.868852,0.947311,1.0,1.0,1.0,1.0,1.0,0.000000,0.042426,0.000361,0.000000,0.306641,0.948586,0.99934,0.000000,1.0,1.0,0.000000,0.215278,0.000000,0.021079,0.000000,0.000000,0.000000,0.000000,0.000000
CPU,0.014336,0.000000,0.0,0.0,0.0,0.0,0.0,0.119360,0.013443,0.018074,0.279356,0.052467,0.000000,0.00000,0.340452,0.0,0.0,0.193746,0.035718,0.195254,0.226151,0.198343,0.035610,0.042145,0.056131,0.039459


In [30]:
# def ran_id_gen(size, chars=string.ascii_uppercase + string.digits):
#     return ''.join(random.choice(chars) for x in range(size))# "Synthetic" nodes data

In [31]:
import random
import string
def ran_id_gen(size, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for x in range(size))

In [32]:
# infra_nodes = pd.DataFrame(columns=['node_id', 'gpu_type', 'energy-efficiency', 'cost-efficiency', 'computational-efficiency'])
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)


In [33]:
infra_nodes = pd.DataFrame.from_dict(input_data)
infra_nodes.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

infra_nodes.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
infra_nodes.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

infra_nodes.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
infra_nodes.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
infra_nodes.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
infra_nodes.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

infra_nodes

,node_id,gpu_type,energy-efficiency,computation-efficiency,cost-efficiency
0,P100_GWSLNE,P100,0.0,0.0,0.0
1,P100_7CUCRY,P100,0.0,0.0,0.0
2,P100_BO2NBI,P100,0.0,0.0,0.0
3,P100_SBFBQ1,P100,0.0,0.0,0.0
4,P100_QCILS2,P100,0.0,0.0,0.0
...,...,...,...,...,...
1892,CPU_FQMAYL,CPU,NaN,NaN,NaN
1893,CPU_CXOUYA,CPU,NaN,NaN,NaN
1894,CPU_U8ZFFS,CPU,NaN,NaN,NaN
1895,CPU_X2GL49,CPU,NaN,NaN,NaN


In [34]:
workloads_baseline_info

,gpu_type_machine_spec,profile
job_name,,
71712e3c961b5bbdb2ce0440,P100,9
8aa3a3a479479aecb41ea324,P100,9
b68f25260271d65caccca8e7,T4,0
8aa3a3a479479aecb41ea324,P100,9
446cd4921e0f00ab5ce1c2ea,P100,9
...,...,...
e49cae3b33b5c205ed8542b0,T4,9
23f6e20075e637b0976b4ac8,T4,-1
bf7ee12301073cc75cba61a2,T4,9


In [35]:
workloads_baseline_info.sample(10, random_state=1234)

,gpu_type_machine_spec,profile
job_name,,
aa3d7d89fbbcd16c2997a1aa,MISC,9
f7a8f8989631b50b0c9d1223,MISC,23
243eea3ba2bfbad7419c2312,T4,1
893a2807e4789e625a872a11,MISC,19
4cfefc5d0d25c3b14380c566,P100,10
a8cbb876f5e3a8bb8787d233,P100,-1
ea2d6955dc3b7717240b4a71,MISC,14
91a00ba3fdcbe5c51e169746,MISC,-1
1d27635bd57a59d543e9bfcc,P100,8


## RANDOM SAMPLING - Example

In [ ]:
# Set sampling size
total_nodes = len(infra_nodes.index)
sampling_perc = 2

nodes_to_sample = int(total_nodes*sampling_perc/100)

# Randomly select nodes
selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=1234)
sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
sel_nodes_list

In [ ]:
expecte_output = ['MISC'] * len(sel_nodes_list)

result = f1_score(expecte_output, sel_nodes_list, average = 'micro')
result

## INTELLIGENT SAMPLING - Example

In [ ]:
incoming_profile = 9
expected_nodes = np.asarray(gpu_given_profile[incoming_profile].to_list())
expected_nodes = (nodes_to_sample * expected_nodes).astype('int')
expected_nodes

In [ ]:
gpu_given_profile.index

In [ ]:
IS_selected_nodes_1 = infra_nodes.loc[infra_nodes['gpu_type']=='V100'].sample(2)
IS_selected_nodes_2 = infra_nodes.loc[infra_nodes['gpu_type']=='P100'].sample(28)
IS_selected_nodes_3 = infra_nodes.loc[infra_nodes['gpu_type']=='MISC'].sample(3)
IS_selected_nodes_4 = infra_nodes.loc[infra_nodes['gpu_type']=='V100M32'].sample(1)
IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4])

In [ ]:
IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

expected_output = ['MISC'] * len(IS_selected_list)

result = f1_score(expected_output, IS_selected_list, average = 'micro')
result

# Random sample evaluation
### Selection of parameters

In [ ]:
results_fscore = list()
results_acc = list()

In [ ]:
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]
seed_id = 9

trial_results_fscore = list()
trial_results_acc = list()

total_nodes = len(infra_nodes.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

### Selecting Workloads

In [ ]:
incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[seed_id])

nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)

### Loop over incoming workloads

In [ ]:
for index, row in incoming_workloads.iterrows():
    selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=repetition_seeds[seed_id])
    sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
    expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
    try:    
        f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
    except(ValueError):
        print(row)
    else:
        trial_results_fscore.append(f1_result)
    try:    
        acc_result = accuracy_score(expected_output, sel_nodes_list)
    except(ValueError):
        print(row)
    else:
        trial_results_acc.append(acc_result)
    
        

In [ ]:
results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
results_acc.append(sum(trial_results_acc)/len(trial_results_acc))

In [ ]:
#time = [6.51,6.52,6.53,6.50,6.52,6.50,6.51,6.49,6.49,6.43]
print(results_fscore)
print(results_acc)

# SINGLE LOOP - RANDOM SAMPING

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes.index)
nodes_sampling_perc = 1
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 1

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

# Intelligent samping evaluation
### 5% of the workloads

In [ ]:
incoming_workloads=workloads_baseline_info.sample(int(0.05*len(workloads_baseline_info.index)), random_state=1234)

results_list_IS_profile = list()
results_list_IS_no_profile = list()

print(len(incoming_workloads.index))

### Number of nodes to sample - 5%

In [ ]:
total_nodes = len(infra_nodes.index)
sampling_perc = 5

nodes_to_sample = int(total_nodes*sampling_perc/100)

print(nodes_to_sample)

### Loop over incoming workloads

In [ ]:
for index, row in incoming_workloads.iterrows():
    
    if row['profile'] == -1:
        selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=1234)
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            results_list_IS_no_profile.append(result)
    else:
        perc_sampling = gpu_given_profile.loc[:, row['profile']]
        
        IS_selected_nodes_1 = infra_nodes.loc[infra_nodes['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample))
        IS_selected_nodes_2 = infra_nodes.loc[infra_nodes['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample))
        IS_selected_nodes_3 = infra_nodes.loc[infra_nodes['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample))
        IS_selected_nodes_4 = infra_nodes.loc[infra_nodes['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample))
        IS_selected_nodes_5 = infra_nodes.loc[infra_nodes['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample))
        IS_selected_nodes_6 = infra_nodes.loc[infra_nodes['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample))
        IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])
        
        IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
        
        expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)
        
        try:    
            result = f1_score(expected_output, IS_selected_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            results_list_IS_profile.append(result)    

In [ ]:
print(sum(results_list_IS_no_profile)/len(results_list_IS_no_profile))

In [ ]:
print(sum(results_list_IS_profile)/len(results_list_IS_profile))

# Intelligent samping evaluation - Equivalent types
### 5% of the workloads

In [ ]:
incoming_workloads=workloads_baseline_info.sample(int(0.05*len(workloads_baseline_info.index)), random_state=1234)

results_list_IS_profile = list()
results_list_IS_no_profile = list()

print(len(incoming_workloads.index))

### Number of nodes to sample - 5%

In [ ]:
total_nodes = len(infra_nodes.index)
sampling_perc = 5

nodes_to_sample = int(total_nodes*sampling_perc/100)

print(nodes_to_sample)

### Loop over incoming workloads

In [ ]:
for index, row in incoming_workloads.iterrows():
    
    if row['profile'] == -1:
        selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=1234)
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            results_list_IS_no_profile.append(result)
    else:
        perc_sampling = gpu_given_profile.loc[:, row['profile']]
        
        IS_selected_nodes_1 = infra_nodes.loc[infra_nodes['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample))
        IS_selected_nodes_2 = infra_nodes.loc[infra_nodes['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample))
        IS_selected_nodes_3 = infra_nodes.loc[infra_nodes['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample))
        IS_selected_nodes_4 = infra_nodes.loc[infra_nodes['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample))
        IS_selected_nodes_5 = infra_nodes.loc[infra_nodes['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample))
        IS_selected_nodes_6 = infra_nodes.loc[infra_nodes['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample))
        IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])
        
        IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
        
        equivalent_selected_list = list()
        expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)
        
        if row['gpu_type_machine_spec'] == 'V100': 
            for val in IS_selected_list:
                if val == 'V100M32':
                    equivalent_selected_list.append('V100')
                else:
                    equivalent_selected_list.append(val)
        elif row['gpu_type_machine_spec'] == 'V100M32':
            for val in IS_selected_list:
                if val == 'V100':
                    equivalent_selected_list.append('V100M32')
                else:
                    equivalent_selected_list.append(val)
        else:
            equivalent_selected_list = IS_selected_list
        
        
        try:    
            result = f1_score(expected_output, equivalent_selected_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            results_list_IS_profile.append(result)    

In [ ]:
print(sum(results_list_IS_no_profile)/len(results_list_IS_no_profile))

In [ ]:
print(sum(results_list_IS_profile)/len(results_list_IS_profile))

# SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 10

infra_nodes_V100 = infra_nodes.loc[infra_nodes['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes.loc[infra_nodes['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes.loc[infra_nodes['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes.loc[infra_nodes['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes.loc[infra_nodes['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes.loc[infra_nodes['gpu_type']=='CPU']


for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])
            
            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'

            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



# SINGLE LOOP - ROUND (red) ROBIN

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_shuffled = infra_nodes.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 1
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes.index)
nodes_sampling_perc = 1
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 1

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

# SINGLE LOOP - LAST RECENTLY used

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 10

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

# SCALE ANALYSIS

In [ ]:
# infra_nodes = pd.DataFrame(columns=['node_id', 'gpu_type', 'energy-efficiency', 'cost-efficiency', 'computational-efficiency'])
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
# gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}

expansion_ration = 100

gpus_dict_expanded = {'P100':798*expansion_ration, 'T4':497*expansion_ration, 'MISC':280*expansion_ration, 'V100M32':135*expansion_ration, 'V100':104*expansion_ration, 'CPU':83*expansion_ration}

input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict_expanded[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)

infra_nodes_expanded = pd.DataFrame.from_dict(input_data)
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

infra_nodes_expanded

## SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
            try:
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
            IS_selected_nodes_5 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

            equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            if row['gpu_type_machine_spec'] == 'V100': 
                for val in IS_selected_list:
                    if val == 'V100M32':
                        equivalent_selected_list.append('V100')
                    else:
                        equivalent_selected_list.append(val)
            elif row['gpu_type_machine_spec'] == 'V100M32':
                for val in IS_selected_list:
                    if val == 'V100':
                        equivalent_selected_list.append('V100M32')
                    else:
                        equivalent_selected_list.append(val)
            else:
                equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, equivalent_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
            try:
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
            IS_selected_nodes_5 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

            equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            if row['gpu_type_machine_spec'] == 'V100': 
                for val in IS_selected_list:
                    if val == 'V100M32':
                        equivalent_selected_list.append('V100')
                    else:
                        equivalent_selected_list.append(val)
            elif row['gpu_type_machine_spec'] == 'V100M32':
                for val in IS_selected_list:
                    if val == 'V100':
                        equivalent_selected_list.append('V100M32')
                    else:
                        equivalent_selected_list.append(val)
            else:
                equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, equivalent_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
            try:
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
            IS_selected_nodes_5 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

            equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            if row['gpu_type_machine_spec'] == 'V100': 
                for val in IS_selected_list:
                    if val == 'V100M32':
                        equivalent_selected_list.append('V100')
                    else:
                        equivalent_selected_list.append(val)
            elif row['gpu_type_machine_spec'] == 'V100M32':
                for val in IS_selected_list:
                    if val == 'V100':
                        equivalent_selected_list.append('V100M32')
                    else:
                        equivalent_selected_list.append(val)
            else:
                equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, equivalent_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100'].sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100'].sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC'].sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
            try:
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32'].sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
            IS_selected_nodes_5 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4'].sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU'].sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())

            equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            if row['gpu_type_machine_spec'] == 'V100': 
                for val in IS_selected_list:
                    if val == 'V100M32':
                        equivalent_selected_list.append('V100')
                    else:
                        equivalent_selected_list.append(val)
            elif row['gpu_type_machine_spec'] == 'V100M32':
                for val in IS_selected_list:
                    if val == 'V100':
                        equivalent_selected_list.append('V100M32')
                    else:
                        equivalent_selected_list.append(val)
            else:
                equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, equivalent_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



## SINGLE LOOP - RANDOM SAMPING

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - ROUND (red) ROBIN

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - LEAST RECENTLY used

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

# SCALE ANALYSIS - Diff IS Implementation

In [ ]:
# infra_nodes = pd.DataFrame(columns=['node_id', 'gpu_type', 'energy-efficiency', 'cost-efficiency', 'computational-efficiency'])
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
# gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}

expansion_ration = 100

gpus_dict_expanded = {'P100':798*expansion_ration, 'T4':497*expansion_ration, 'MISC':280*expansion_ration, 'V100M32':135*expansion_ration, 'V100':104*expansion_ration, 'CPU':83*expansion_ration}

input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict_expanded[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)

infra_nodes_expanded = pd.DataFrame.from_dict(input_data)
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

infra_nodes_expanded

## SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



# INFRA SCALE ANALYSIS

In [ ]:
# infra_nodes = pd.DataFrame(columns=['node_id', 'gpu_type', 'energy-efficiency', 'cost-efficiency', 'computational-efficiency'])
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
# gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}

nodes_to_sample = 100
expansion_ration = 100

gpus_dict_expanded = {'P100':798*expansion_ration, 'T4':497*expansion_ration, 'MISC':280*expansion_ration, 'V100M32':135*expansion_ration, 'V100':104*expansion_ration, 'CPU':83*expansion_ration}

input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict_expanded[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)

infra_nodes_expanded = pd.DataFrame.from_dict(input_data)
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

print(len(infra_nodes_expanded.index))

infra_nodes_expanded

## SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

# total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]

            IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
            IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_3 = infra_nodes_MISC
            try:
                IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_4 = infra_nodes_V100M32
            IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
            try:
                IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
            except(ValueError):
                # print("not enought nodes, taking just all of them")
                IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

#             if row['gpu_type_machine_spec'] == 'V100': 
#                 for val in IS_selected_list:
#                     if val == 'V100M32':
#                         equivalent_selected_list.append('V100')
#                     else:
#                         equivalent_selected_list.append(val)
#             elif row['gpu_type_machine_spec'] == 'V100M32':
#                 for val in IS_selected_list:
#                     if val == 'V100':
#                         equivalent_selected_list.append('V100M32')
#                     else:
#                         equivalent_selected_list.append(val)
#             else:
#                 equivalent_selected_list = IS_selected_list


            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)

## SINGLE LOOP - RANDOM SAMPING

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

# total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - ROUND (red) ROBIN

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = nodes_to_sample # int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - LEAST RECENTLY used

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
        sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[selected_nodes.index, 'seen'] = infra_nodes_weight.loc[selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

# FAST SAMPLING - NO REPLACEMENT + All equivalent types

In [36]:
# infra_nodes = pd.DataFrame(columns=['node_id', 'gpu_type', 'energy-efficiency', 'cost-efficiency', 'computational-efficiency'])
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
# gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}

# nodes_to_sample = 100
expansion_ration = 100

gpus_dict_expanded = {'P100':798*expansion_ration, 'T4':497*expansion_ration, 'MISC':280*expansion_ration, 'V100M32':135*expansion_ration, 'V100':104*expansion_ration, 'CPU':83*expansion_ration}

input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict_expanded[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)

infra_nodes_expanded = pd.DataFrame.from_dict(input_data)
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

print(len(infra_nodes_expanded.index))

infra_nodes_expanded

189700


,node_id,gpu_type
0,P100_R9V1EZ,P100
1,P100_BJ5U0A,P100
2,P100_CNP03E,P100
3,P100_132HCG,P100
4,P100_FWMSBA,P100
...,...,...
189695,CPU_DN7DI4,CPU
189696,CPU_KEFZ1G,CPU
189697,CPU_OXBLLQ,CPU
189698,CPU_QUGO5P,CPU


## SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
#     idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            
            idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
#             selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            is_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
    
            if row['gpu_type_machine_spec'] == 'V100': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
                
            sel_nodes_list = np.asarray(is_selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]
            
            idx_to_sample_1 = random.sample(range(len(infra_nodes_V100.index)), int(perc_sampling['V100']*nodes_to_sample))
            idx_to_sample_2 = random.sample(range(len(infra_nodes_P100.index)), int(perc_sampling['P100']*nodes_to_sample))
            idx_to_sample_3 = random.sample(range(len(infra_nodes_MISC.index)), int(perc_sampling['MISC']*nodes_to_sample))
            idx_to_sample_4 = random.sample(range(len(infra_nodes_V100M32.index)), int(perc_sampling['V100M32']*nodes_to_sample))
            idx_to_sample_5 = random.sample(range(len(infra_nodes_T4.index)), int(perc_sampling['T4']*nodes_to_sample))
            idx_to_sample_6 = random.sample(range(len(infra_nodes_CPU.index)), int(perc_sampling['CPU']*nodes_to_sample))
            
            IS_selected_nodes_1 = infra_nodes_V100.iloc[idx_to_sample_1,:]
            IS_selected_nodes_2 = infra_nodes_P100.iloc[idx_to_sample_2,:]
            IS_selected_nodes_3 = infra_nodes_MISC.iloc[idx_to_sample_3,:]
            IS_selected_nodes_4 = infra_nodes_V100M32.iloc[idx_to_sample_4,:]
            IS_selected_nodes_5 = infra_nodes_T4.iloc[idx_to_sample_5,:]
            IS_selected_nodes_6 = infra_nodes_CPU.iloc[idx_to_sample_6,:]
            

#             IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             IS_selected_nodes_1 = infra_nodes_V100.loc[idx_to_sample,:]
#             IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_3 = infra_nodes_MISC
#             try:
#                 IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_4 = infra_nodes_V100M32
#             IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
#             selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            is_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
                
            if row['gpu_type_machine_spec'] == 'V100': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
                
            sel_nodes_list = np.asarray(is_selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]
            
            idx_to_sample_1 = random.sample(range(len(infra_nodes_V100.index)), int(perc_sampling['V100']*nodes_to_sample))
            idx_to_sample_2 = random.sample(range(len(infra_nodes_P100.index)), int(perc_sampling['P100']*nodes_to_sample))
            idx_to_sample_3 = random.sample(range(len(infra_nodes_MISC.index)), int(perc_sampling['MISC']*nodes_to_sample))
            idx_to_sample_4 = random.sample(range(len(infra_nodes_V100M32.index)), int(perc_sampling['V100M32']*nodes_to_sample))
            idx_to_sample_5 = random.sample(range(len(infra_nodes_T4.index)), int(perc_sampling['T4']*nodes_to_sample))
            idx_to_sample_6 = random.sample(range(len(infra_nodes_CPU.index)), int(perc_sampling['CPU']*nodes_to_sample))
            
            IS_selected_nodes_1 = infra_nodes_V100.iloc[idx_to_sample_1,:]
            IS_selected_nodes_2 = infra_nodes_P100.iloc[idx_to_sample_2,:]
            IS_selected_nodes_3 = infra_nodes_MISC.iloc[idx_to_sample_3,:]
            IS_selected_nodes_4 = infra_nodes_V100M32.iloc[idx_to_sample_4,:]
            IS_selected_nodes_5 = infra_nodes_T4.iloc[idx_to_sample_5,:]
            IS_selected_nodes_6 = infra_nodes_CPU.iloc[idx_to_sample_6,:]
            

#             IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             IS_selected_nodes_1 = infra_nodes_V100.loc[idx_to_sample,:]
#             IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_3 = infra_nodes_MISC
#             try:
#                 IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_4 = infra_nodes_V100M32
#             IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
#             selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            is_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
                
            if row['gpu_type_machine_spec'] == 'V100': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
                
            sel_nodes_list = np.asarray(is_selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]
            
            idx_to_sample_1 = random.sample(range(len(infra_nodes_V100.index)), int(perc_sampling['V100']*nodes_to_sample))
            idx_to_sample_2 = random.sample(range(len(infra_nodes_P100.index)), int(perc_sampling['P100']*nodes_to_sample))
            idx_to_sample_3 = random.sample(range(len(infra_nodes_MISC.index)), int(perc_sampling['MISC']*nodes_to_sample))
            idx_to_sample_4 = random.sample(range(len(infra_nodes_V100M32.index)), int(perc_sampling['V100M32']*nodes_to_sample))
            idx_to_sample_5 = random.sample(range(len(infra_nodes_T4.index)), int(perc_sampling['T4']*nodes_to_sample))
            idx_to_sample_6 = random.sample(range(len(infra_nodes_CPU.index)), int(perc_sampling['CPU']*nodes_to_sample))
            
            IS_selected_nodes_1 = infra_nodes_V100.iloc[idx_to_sample_1,:]
            IS_selected_nodes_2 = infra_nodes_P100.iloc[idx_to_sample_2,:]
            IS_selected_nodes_3 = infra_nodes_MISC.iloc[idx_to_sample_3,:]
            IS_selected_nodes_4 = infra_nodes_V100M32.iloc[idx_to_sample_4,:]
            IS_selected_nodes_5 = infra_nodes_T4.iloc[idx_to_sample_5,:]
            IS_selected_nodes_6 = infra_nodes_CPU.iloc[idx_to_sample_6,:]
            

#             IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             IS_selected_nodes_1 = infra_nodes_V100.loc[idx_to_sample,:]
#             IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_3 = infra_nodes_MISC
#             try:
#                 IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_4 = infra_nodes_V100M32
#             IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
#             selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            is_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
            
            if row['gpu_type_machine_spec'] == 'V100': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
    
            sel_nodes_list = np.asarray(is_selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]
            
            if len(infra_nodes_V100.index) > 0:
                idx_to_sample_1 = random.choices(range(len(infra_nodes_V100.index)), k = int(perc_sampling['V100']*nodes_to_sample))
            else:
                idx_to_sample_1 = []
            if len(infra_nodes_MISC.index) > 0:
                idx_to_sample_2 = random.choices(range(len(infra_nodes_P100.index)), k = int(perc_sampling['P100']*nodes_to_sample))
            else:
                idx_to_sample_2 = []
            if len(infra_nodes_MISC.index) > 0:
                idx_to_sample_3 = random.choices(range(len(infra_nodes_MISC.index)), k = int(perc_sampling['MISC']*nodes_to_sample))
            else:
                idx_to_sample_3 = []
            if len(infra_nodes_V100M32.index) > 0:
                idx_to_sample_4 = random.choices(range(len(infra_nodes_V100M32.index)), k = int(perc_sampling['V100M32']*nodes_to_sample))
            else:
                idx_to_sample_4 = []
            if len(infra_nodes_T4.index) > 0:
                idx_to_sample_5 = random.choices(range(len(infra_nodes_T4.index)), k = int(perc_sampling['T4']*nodes_to_sample))
            else:
                idx_to_sample_5 = []
            if len(infra_nodes_CPU.index) > 0:
                idx_to_sample_6 = random.choices(range(len(infra_nodes_CPU.index)), k = int(perc_sampling['CPU']*nodes_to_sample))
            else:
                idx_to_sample_6 = []
            
            IS_selected_nodes_1 = infra_nodes_V100.iloc[idx_to_sample_1,:]
            IS_selected_nodes_2 = infra_nodes_P100.iloc[idx_to_sample_2,:]
            IS_selected_nodes_3 = infra_nodes_MISC.iloc[idx_to_sample_3,:]
            IS_selected_nodes_4 = infra_nodes_V100M32.iloc[idx_to_sample_4,:]
            IS_selected_nodes_5 = infra_nodes_T4.iloc[idx_to_sample_5,:]
            IS_selected_nodes_6 = infra_nodes_CPU.iloc[idx_to_sample_6,:]
            

#             IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             IS_selected_nodes_1 = infra_nodes_V100.loc[idx_to_sample,:]
#             IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_3 = infra_nodes_MISC
#             try:
#                 IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_4 = infra_nodes_V100M32
#             IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



## SINGLE LOOP - RANDOM SAMPING

In [40]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
#         selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        rs_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
        
        if row['gpu_type_machine_spec'] == 'V100': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(rs_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

gpu_type_machine_spec    NaN
profile                   23
Name: e3bee45f2156a564be9ed783, dtype: object
gpu_type_machine_spec    NaN
profile                   20
Name: fda95b4ded2d3e9a482c1245, dtype: object
gpu_type_machine_spec    NaN
profile                   -1
Name: 7bff47b536aa9b2b532fa3c3, dtype: object
gpu_type_machine_spec    NaN
profile                   25
Name: 9bdda1a44c4a3b767f5967ff, dtype: object
gpu_type_machine_spec    NaN
profile                   19
Name: 7057dbd9a884d68937346ac6, dtype: object
gpu_type_machine_spec    NaN
profile                   14
Name: 0b5918267540c51f95fe79d2, dtype: object
gpu_type_machine_spec    NaN
profile                    8
Name: e3b3cc14b2a6b3b331c3b140, dtype: object
gpu_type_machine_spec    NaN
profile                    0
Name: 7b306c25c1464cb00e78b83d, dtype: object
gpu_type_machine_spec    NaN
profile                   23
Name: ecdfb37ff64a7eb23f42237b, dtype: object
gpu_type_machine_spec    NaN
profile                    9
Name: 

In [39]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
#         selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        rs_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
        
        if row['gpu_type_machine_spec'] == 'V100': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(rs_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

gpu_type_machine_spec    NaN
profile                   23
Name: e3bee45f2156a564be9ed783, dtype: object
gpu_type_machine_spec    NaN
profile                   20
Name: fda95b4ded2d3e9a482c1245, dtype: object
gpu_type_machine_spec    NaN
profile                   -1
Name: 7bff47b536aa9b2b532fa3c3, dtype: object
gpu_type_machine_spec    NaN
profile                   25
Name: 9bdda1a44c4a3b767f5967ff, dtype: object
gpu_type_machine_spec    NaN
profile                   19
Name: 7057dbd9a884d68937346ac6, dtype: object
gpu_type_machine_spec    NaN
profile                   14
Name: 0b5918267540c51f95fe79d2, dtype: object
gpu_type_machine_spec    NaN
profile                    8
Name: e3b3cc14b2a6b3b331c3b140, dtype: object
gpu_type_machine_spec    NaN
profile                    0
Name: 7b306c25c1464cb00e78b83d, dtype: object
gpu_type_machine_spec    NaN
profile                   23
Name: ecdfb37ff64a7eb23f42237b, dtype: object
gpu_type_machine_spec    NaN
profile                    9
Name: 

In [38]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
#         idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
        idx_to_sample = np.random.choice(total_nodes, nodes_to_sample)
#         selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        rs_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
        
        if row['gpu_type_machine_spec'] == 'V100': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(rs_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

gpu_type_machine_spec    NaN
profile                   23
Name: e3bee45f2156a564be9ed783, dtype: object
gpu_type_machine_spec    NaN
profile                   20
Name: fda95b4ded2d3e9a482c1245, dtype: object
gpu_type_machine_spec    NaN
profile                   -1
Name: 7bff47b536aa9b2b532fa3c3, dtype: object
gpu_type_machine_spec    NaN
profile                   25
Name: 9bdda1a44c4a3b767f5967ff, dtype: object
gpu_type_machine_spec    NaN
profile                   19
Name: 7057dbd9a884d68937346ac6, dtype: object
gpu_type_machine_spec    NaN
profile                   14
Name: 0b5918267540c51f95fe79d2, dtype: object
gpu_type_machine_spec    NaN
profile                    8
Name: e3b3cc14b2a6b3b331c3b140, dtype: object
gpu_type_machine_spec    NaN
profile                    0
Name: 7b306c25c1464cb00e78b83d, dtype: object
gpu_type_machine_spec    NaN
profile                   23
Name: ecdfb37ff64a7eb23f42237b, dtype: object
gpu_type_machine_spec    NaN
profile                    9
Name: 

In [37]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
#         idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
        idx_to_sample = np.random.choice(total_nodes, nodes_to_sample)
#         selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        rs_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
        
        if row['gpu_type_machine_spec'] == 'V100': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(rs_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

gpu_type_machine_spec    NaN
profile                   23
Name: e3bee45f2156a564be9ed783, dtype: object
gpu_type_machine_spec    NaN
profile                   20
Name: fda95b4ded2d3e9a482c1245, dtype: object
gpu_type_machine_spec    NaN
profile                   -1
Name: 7bff47b536aa9b2b532fa3c3, dtype: object
gpu_type_machine_spec    NaN
profile                   25
Name: 9bdda1a44c4a3b767f5967ff, dtype: object
gpu_type_machine_spec    NaN
profile                   19
Name: 7057dbd9a884d68937346ac6, dtype: object
gpu_type_machine_spec    NaN
profile                   14
Name: 0b5918267540c51f95fe79d2, dtype: object
gpu_type_machine_spec    NaN
profile                    8
Name: e3b3cc14b2a6b3b331c3b140, dtype: object
gpu_type_machine_spec    NaN
profile                    0
Name: 7b306c25c1464cb00e78b83d, dtype: object
gpu_type_machine_spec    NaN
profile                   23
Name: ecdfb37ff64a7eb23f42237b, dtype: object
gpu_type_machine_spec    NaN
profile                    9
Name: 

## SINGLE LOOP - ROUND (red) ROBIN

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
#         selected_nodes = RR_selected_nodes.copy()
            
        if row['gpu_type_machine_spec'] == 'V100': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        if row['gpu_type_machine_spec'] == 'V100': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        if row['gpu_type_machine_spec'] == 'V100': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
            
        if row['gpu_type_machine_spec'] == 'V100': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - LEAST RECENTLY used

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        lr_selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
#         sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
#         lr_selected_nodes = selected_nodes.copy()
    
        if row['gpu_type_machine_spec'] == 'V100': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(lr_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
#         expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] = infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[lr_selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[lr_selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 5
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        lr_selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
#         lr_selected_nodes = selected_nodes.copy()
#         sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        
        if row['gpu_type_machine_spec'] == 'V100': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(lr_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
#         expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] = infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[lr_selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[lr_selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 10
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        lr_selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
#         lr_selected_nodes = selected_nodes.copy()
#         sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        
        if row['gpu_type_machine_spec'] == 'V100': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(lr_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
#         expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] = infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[lr_selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[lr_selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 25
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

    nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        lr_selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
#         lr_selected_nodes = selected_nodes.copy()
#         sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
        
        if row['gpu_type_machine_spec'] == 'V100': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(lr_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
#         expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] = infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[lr_selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[lr_selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)

# INFRA SCALE ANALYSIS V2

In [ ]:
gpu_types = ['P100', 'T4', 'MISC', 'V100M32', 'V100', 'CPU']
# gpus_dict = {'P100':798, 'T4':497, 'MISC':280, 'V100M32':135, 'V100':104, 'CPU':83}

nodes_to_sample = 100
expansion_ration = 100

gpus_dict_expanded = {'P100':798*expansion_ration, 'T4':497*expansion_ration, 'MISC':280*expansion_ration, 'V100M32':135*expansion_ration, 'V100':104*expansion_ration, 'CPU':83*expansion_ration}

input_data = {'node_id':list(), 'gpu_type':list()}
for val in gpu_types:
    for ii in range(gpus_dict_expanded[val]):
        input_data['node_id'].append(val + '_' + ran_id_gen(6))
        input_data['gpu_type'].append(val)

infra_nodes_expanded = pd.DataFrame.from_dict(input_data)
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'energy-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'energy-efficiency'] = 1

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'computation-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'computation-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'computation-efficiency'] = 0

# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'MISC', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'P100', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100', 'cost-efficiency'] = 1
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'V100M32', 'cost-efficiency'] = 0
# infra_nodes_expanded.loc[infra_nodes['gpu_type'] == 'T4', 'cost-efficiency'] = 0

print(len(infra_nodes_expanded.index))

infra_nodes_expanded

## SINGLE LOOP - INTELLIGENT SAMPLING

In [ ]:
results_fscore_profile = list()
results_fscore_no_profile = list()
exe_time = list()

infra_nodes_V100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100']
infra_nodes_P100 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='P100']
infra_nodes_MISC = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='MISC']
infra_nodes_V100M32 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='V100M32']
infra_nodes_T4 = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='T4']
infra_nodes_CPU = infra_nodes_expanded.loc[infra_nodes_expanded['gpu_type']=='CPU']


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

# total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore_profile = list()
    trial_results_fscore_no_profile = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
#     idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        if row['profile'] == -1:
            
            idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
#             selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
            is_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
    
            if row['gpu_type_machine_spec'] == 'V100': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                is_selected_nodes.loc[is_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
                
            sel_nodes_list = np.asarray(is_selected_nodes['gpu_type'].to_list())
            expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
            try:    
                result = f1_score(expected_output, sel_nodes_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_no_profile.append(result)
        else:
            perc_sampling = gpu_given_profile.loc[:, row['profile']]
            
            idx_to_sample_1 = random.sample(range(len(infra_nodes_V100.index)), int(perc_sampling['V100']*nodes_to_sample))
            idx_to_sample_2 = random.sample(range(len(infra_nodes_P100.index)), int(perc_sampling['P100']*nodes_to_sample))
            idx_to_sample_3 = random.sample(range(len(infra_nodes_MISC.index)), int(perc_sampling['MISC']*nodes_to_sample))
            idx_to_sample_4 = random.sample(range(len(infra_nodes_V100M32.index)), int(perc_sampling['V100M32']*nodes_to_sample))
            idx_to_sample_5 = random.sample(range(len(infra_nodes_T4.index)), int(perc_sampling['T4']*nodes_to_sample))
            idx_to_sample_6 = random.sample(range(len(infra_nodes_CPU.index)), int(perc_sampling['CPU']*nodes_to_sample))
            
            IS_selected_nodes_1 = infra_nodes_V100.iloc[idx_to_sample_1,:]
            IS_selected_nodes_2 = infra_nodes_P100.iloc[idx_to_sample_2,:]
            IS_selected_nodes_3 = infra_nodes_MISC.iloc[idx_to_sample_3,:]
            IS_selected_nodes_4 = infra_nodes_V100M32.iloc[idx_to_sample_4,:]
            IS_selected_nodes_5 = infra_nodes_T4.iloc[idx_to_sample_5,:]
            IS_selected_nodes_6 = infra_nodes_CPU.iloc[idx_to_sample_6,:]
            

#             IS_selected_nodes_1 = infra_nodes_V100.sample(int(perc_sampling['V100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             IS_selected_nodes_1 = infra_nodes_V100.loc[idx_to_sample,:]
#             IS_selected_nodes_2 = infra_nodes_P100.sample(int(perc_sampling['P100']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_3 = infra_nodes_MISC.sample(int(perc_sampling['MISC']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_3 = infra_nodes_MISC
#             try:
#                 IS_selected_nodes_4 = infra_nodes_V100M32.sample(int(perc_sampling['V100M32']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_4 = infra_nodes_V100M32
#             IS_selected_nodes_5 = infra_nodes_T4.sample(int(perc_sampling['T4']*nodes_to_sample), random_state=repetition_seeds[rep])
#             try:
#                 IS_selected_nodes_6 = infra_nodes_CPU.sample(int(perc_sampling['CPU']*nodes_to_sample), random_state=repetition_seeds[rep])
#             except(ValueError):
#                 IS_selected_nodes_6 = infra_nodes_CPU
                
            IS_selected_nodes = pd.concat([IS_selected_nodes_1, IS_selected_nodes_2, IS_selected_nodes_3, IS_selected_nodes_4, IS_selected_nodes_5, IS_selected_nodes_6])

            if row['gpu_type_machine_spec'] == 'V100': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
            if row['gpu_type_machine_spec'] == 'V100M32': 
                IS_selected_nodes.loc[IS_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
            IS_selected_list = np.asarray(IS_selected_nodes['gpu_type'].to_list())
            
#             equivalent_selected_list = list()
            expected_output = [row['gpu_type_machine_spec']] * len(IS_selected_list)

            try:    
                result = f1_score(expected_output, IS_selected_list, average = 'micro')
            except(ValueError):
                print(row)
            else:
                trial_results_fscore_profile.append(result)    

    elapsed_time = time.time() - st

    results_fscore_profile.append(sum(trial_results_fscore_profile)/len(trial_results_fscore_profile))
    results_fscore_no_profile.append(sum(trial_results_fscore_no_profile)/len(trial_results_fscore_no_profile))
    exe_time.append(elapsed_time)
    
print(results_fscore_profile)
print(results_fscore_no_profile)
print(exe_time)



## SINGLE LOOP - RANDOM SAMPING

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

# total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):
    
    random.seed(repetition_seeds[rep])

    trial_results_fscore = list()
    trial_results_acc = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        idx_to_sample = random.sample(range(len(infra_nodes_expanded.index)), nodes_to_sample)
#         selected_nodes = infra_nodes_expanded.sample(n=nodes_to_sample, random_state=repetition_seeds[rep])
        rs_selected_nodes = infra_nodes_expanded.iloc[idx_to_sample,:].copy()
        
        if row['gpu_type_machine_spec'] == 'V100': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            rs_selected_nodes.loc[rs_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(rs_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - ROUND (red) ROBIN

In [ ]:
results_fscore = list()
exe_time = list()

# Shuffle dataset:
infra_nodes_expanded_shuffled = infra_nodes_expanded.sample(frac=1, random_state = 1234).reset_index(drop=True)


repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

# total_nodes = len(infra_nodes_expanded.index)
# nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()
    
    ii = 0
    sampled_nodes = nodes_to_sample #int(total_nodes * nodes_sampling_perc / 100)
    sn_ii = sampled_nodes
    
    for index, row in incoming_workloads.iterrows():
        RR_selected_nodes = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:].copy()
        if len(RR_selected_nodes) < sampled_nodes:
            ii = 0
            sn_ii = sampled_nodes - len(RR_selected_nodes)
            selected_nodes_extra = infra_nodes_expanded_shuffled.loc[ii:sn_ii,:]
            RR_selected_nodes = pd.concat([RR_selected_nodes, selected_nodes_extra])
#         selected_nodes = RR_selected_nodes.copy()
            
        if row['gpu_type_machine_spec'] == 'V100': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            RR_selected_nodes.loc[RR_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
            
        sel_nodes_list = np.asarray(RR_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        ii = sn_ii
        sn_ii = sn_ii + nodes_to_sample
        if sn_ii > total_nodes:
            sn_ii = total_nodes
            
        

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
print(results_fscore)
print(exe_time)

## SINGLE LOOP - LEAST RECENTLY used

In [ ]:
results_fscore = list()
exe_time = list()

repetitions = 10
repetition_seeds = [1234,1435,4861,8964,1869,9781,3657,6278,2057,8524]

infra_nodes_weight = infra_nodes_expanded.copy()
infra_nodes_weight['weight'] = 1
infra_nodes_weight['seen'] = 1

total_nodes = len(infra_nodes_expanded.index)
nodes_sampling_perc = 2
total_workloads = len(workloads_baseline_info.index)
wl_sampling_perc = 5

for rep in range(repetitions):

    trial_results_fscore = list()

    incoming_workloads=workloads_baseline_info.sample(int((wl_sampling_perc/100)*total_workloads), random_state=repetition_seeds[rep])

#     nodes_to_sample = int(total_nodes*nodes_sampling_perc/100)
    
    st = time.time()

    for index, row in incoming_workloads.iterrows():
        lr_selected_nodes = infra_nodes_weight.sample(n=nodes_to_sample, random_state=repetition_seeds[rep], weights=infra_nodes_weight['weight'])
#         sel_nodes_list = np.asarray(selected_nodes['gpu_type'].to_list())
#         lr_selected_nodes = selected_nodes.copy()
    
        if row['gpu_type_machine_spec'] == 'V100': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100M32'] = 'V100'
        if row['gpu_type_machine_spec'] == 'V100M32': 
            lr_selected_nodes.loc[lr_selected_nodes['gpu_type'] == 'V100'] = 'V100M32'
        
        sel_nodes_list = np.asarray(lr_selected_nodes['gpu_type'].to_list())
        expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        
#         expected_output = [row['gpu_type_machine_spec']] * len(sel_nodes_list)
        try:    
            f1_result = f1_score(expected_output, sel_nodes_list, average = 'micro')
        except(ValueError):
            print(row)
        else:
            trial_results_fscore.append(f1_result)
            
        infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] = infra_nodes_weight.loc[lr_selected_nodes.index, 'seen'] + 1
        infra_nodes_weight.loc[lr_selected_nodes.index, 'weight'] = 1 / infra_nodes_weight.loc[lr_selected_nodes.index, 'seen']

    elapsed_time = time.time() - st

    results_fscore.append(sum(trial_results_fscore)/len(trial_results_fscore))
    exe_time.append(elapsed_time)
    
    infra_nodes_weight['weight'] = 1
    infra_nodes_weight['seen'] = 1
    
print(results_fscore)
print(exe_time)